### Goal
This file perform the pre-processing step on the dataset to
keep only 5 entity's labels and non-entity label.

Included entities: PERSON(PER), ORGANIZATION(ORG), LOCATION(LOC), DISEASES(DIS),
ANIMAL(ANIM), and non-entity(O)

In [1]:
!pip install --upgrade huggingface_hub

packages we need to use HuggingFace API

In [2]:
!pip install transformers datasets tokenizers seqeval -q

In [3]:
import numpy as np
import datasets
from transformers import BertTokenizerFast                        # We need bert tokenizer
from transformers import DataCollatorForTokenClassification       # DataCollator for some tasks like data augmentation in NLP task (token classification)
from transformers import AutoModelForTokenClassification          # For loading model in the memory.

### login to huggingface

In [4]:
from huggingface_hub import notebook_login
notebook_login()

### Legal labels:PERSON(PER), ORGANIZATION(ORG), LOCATION(LOC), DISEASES(DIS),
ANIMAL(ANIM):
{"O": 0, "B-PER": 1,"I-PER": 2, "B-ORG": 3,"I-ORG": 4,"B-LOC": 5,"I-LOC": 6,"B-ANIM": 7,"I-ANIM": 8,"B-DIS": 13,"I-DIS": 14,
    
  }

In [5]:
legal_labels = [0,1,2,3,4,5,6,7,8,13,14]

### Loading Dataset

In [6]:
en_dataset = datasets.load_dataset('MaryDatascientist/en_dataset')

In [7]:
en_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32908
    })
})

### Replace the labels we do not want to predict with 0
###  Map function

In [8]:
def keep_legal_tags(row):
  row['ner_tags']=list( map(lambda x: x if x in legal_labels else 0,row['ner_tags']))
  return row

def set_continuous_tags(row):
  row['ner_tags']=list( map(lambda x: 9 if x==13   else(10 if x==14 else x) ,row['ner_tags']))
  return row

In [9]:
from datasets import DatasetDict
import time
#exclude illegal tags
less_tags_tr = en_dataset['train'].map( keep_legal_tags )
less_tags_val = en_dataset['validation'].map( keep_legal_tags )
less_tags_test = en_dataset['test'].map( keep_legal_tags )

#map 13 and 14 to 9 and 10
less_tags_train_ds= less_tags_tr.map( set_continuous_tags )
less_tags_val_ds = less_tags_val.map( set_continuous_tags )
less_tags_test_ds = less_tags_test.map( set_continuous_tags )

less_tags_dataset = DatasetDict({'train':less_tags_train_ds, 'validation':less_tags_val_ds,'test':less_tags_test_ds })

In [10]:
less_tags_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32908
    })
})

In [11]:
less_tags_dataset.push_to_hub('MaryDatascientist/ten_tags_dataset')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/263 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

Well Done!

